# Fake News Detection 
1. News Data
2. Data PreProcessing ( Text => Numerical Meaningful Data )
3. Train _Test_Split
4. Using Logistic Regression Model
5. Evaluation of the model 

In [56]:
#Import dependecies 
import numpy as np 
import pandas as pd
import re #Manipluate Strings 
from nltk.corpus import stopwords #Words that doesn't affect the result like 'The, a ' stopwords
from nltk.stem.porter import PorterStemmer # Porter stemmer : process for removing suffixes from words in English
from sklearn.feature_extraction.text import TfidfVectorizer #Text => Numerical data 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [57]:
import nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [58]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [59]:
#Data Preprocessing
news_df = pd.read_csv('/content/train.csv')

In [60]:
news_df.shape

(20800, 5)

In [61]:
news_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [62]:
news_df.info

<bound method DataFrame.info of           id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2      

In [63]:
#Missing Values Check 
news_df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [64]:
#Replacing the missing values with empty string 
news_df = news_df.fillna('')

In [65]:
news_df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

# Predictions based on 'Title' + 'author name'

In [66]:
#Combining author name + title
news_df['content'] = news_df['author'] + ' ' + news_df['title']
news_df.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [67]:
 #Using stemmer : reducing words (removing prefixes and suffixes)
port_stem  = PorterStemmer()

In [68]:
#Define the stemming method
def stemming (content) : 
  stem_content  = re.sub('[^a-zA-Z]',' ',content)
  stem_content = stem_content.lower()
  stem_content = stem_content.split()
  stem_content = [port_stem.stem(word) for word in stem_content if not word in stopwords.words('english')]
  stem_content = ' '.join(stem_content)
  return stem_content

In [69]:
news_df['content'] = news_df['content'].apply(stemming)

In [70]:
print(news_df['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [71]:
#Separate data and label 
X= news_df['content'].values
Y = news_df['label'].values

In [ ]:
#Vectorize the text data 
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X= vectorizer.transform(X)

In [84]:
#train_test_data split 
X_train , X_test , Y_train , Y_test = train_test_split(X,Y,train_size=.8,stratify=Y,random_state=2)

In [ ]:
print(X_train)

In [74]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(16640,)
(4160,)
(16640,)
(4160,)


In [86]:
#model declare 
model = LogisticRegression()

In [87]:
model.fit(X_train,Y_train)

LogisticRegression()

In [90]:
#Predictions on training data 
predict_train = model.predict(X_train)
#Accuracy score on training data 
print('Accuracy Score on Training Data = ',accuracy_score(Y_train,predict_train))

Accuracy Score on Training Data =  0.9865985576923076


In [91]:
#Predictions on test data 
predict_test= model.predict(X_test)
#Accuracy score on test data 
print('Accuracy Score on Test Data = ',accuracy_score(Y_test,predict_test))

Accuracy Score on Test Data =  0.9790865384615385


In [105]:
# #Set up the system 
# author = input('Please introduce the author of the news : ')
# title = input('Please introduce the title of the news : ')
# x_content = author + ' ' + title
# x_content = np.array(stemming(x_content))
# x_content_vect  = vectorizer.transform(x_content)
# print(x_content)
# prediction = model.predict(x_content)

# if (prediction[0]==0) : 
#   print('This news is valid ')
# else : 
#   print('This news is fake ')